## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/Weather_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,Taltal,CL,-25.4000,-70.4833,69.75,71,24,6.87,2022-03-13 19:45:13
1,1,Kristiinankaupunki,FI,62.2743,21.3760,26.42,91,48,5.35,2022-03-13 19:45:14
2,2,Lethem,GY,3.3803,-59.7968,92.19,43,100,9.01,2022-03-13 19:45:14
3,3,Hermanus,ZA,-34.4187,19.2345,60.80,61,100,15.28,2022-03-13 19:43:30
4,4,Hambantota,LK,6.1241,81.1185,77.67,80,75,9.44,2022-03-13 19:45:15


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,Taltal,CL,-25.4000,-70.4833,69.75,71,24,6.87,2022-03-13 19:45:13
3,3,Hermanus,ZA,-34.4187,19.2345,60.80,61,100,15.28,2022-03-13 19:43:30
6,6,Chuy,UY,-33.6971,-53.4616,69.58,55,4,9.08,2022-03-13 19:45:15
7,7,Port Elizabeth,ZA,-33.9180,25.5701,64.69,94,75,16.11,2022-03-13 19:45:16
8,8,Busselton,AU,-33.6500,115.3333,63.68,88,11,7.99,2022-03-13 19:45:16
11,11,Mar Del Plata,AR,-38.0023,-57.5575,69.82,81,0,11.50,2022-03-13 19:34:53
12,12,Bredasdorp,ZA,-34.5322,20.0403,61.92,59,100,12.66,2022-03-13 19:45:17
13,13,Semnan,IR,35.5729,53.3971,58.84,33,0,2.30,2022-03-13 19:45:17
14,14,Ribeira Grande,PT,38.5167,-28.7000,57.60,54,40,4.61,2022-03-13 19:45:18
17,17,Marawi,PH,7.9986,124.2928,68.95,94,98,2.08,2022-03-13 19:45:19


In [5]:
# 4a. Determine if there are any empty rows.
(preferred_cities_df).isnull

<bound method DataFrame.isnull of      City_ID              City Country      Lat       Lon  Max Temp  Humidity  \
0          0            Taltal      CL -25.4000  -70.4833     69.75        71   
3          3          Hermanus      ZA -34.4187   19.2345     60.80        61   
6          6              Chuy      UY -33.6971  -53.4616     69.58        55   
7          7    Port Elizabeth      ZA -33.9180   25.5701     64.69        94   
8          8         Busselton      AU -33.6500  115.3333     63.68        88   
..       ...               ...     ...      ...       ...       ...       ...   
680      680             Asyut      EG  27.1810   31.1837     55.81        45   
684      684          Mosquera      CO   4.7059  -74.2302     60.67        80   
692      692            Cobija      BO -11.0267  -68.7692     73.98        93   
702      702             Sucua      EC  -2.4667  -78.1667     72.34        88   
706      706  Praia Da Vitoria      PT  38.7333  -27.0667     55.76        

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()
clean_df = preferred_cities_df.copy()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lon"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lon,Hotel Name
0,Taltal,CL,69.75,-25.4000,-70.4833,
3,Hermanus,ZA,60.80,-34.4187,19.2345,
6,Chuy,UY,69.58,-33.6971,-53.4616,
7,Port Elizabeth,ZA,64.69,-33.9180,25.5701,
8,Busselton,AU,63.68,-33.6500,115.3333,
11,Mar Del Plata,AR,69.82,-38.0023,-57.5575,
12,Bredasdorp,ZA,61.92,-34.5322,20.0403,
13,Semnan,IR,58.84,35.5729,53.3971,
14,Ribeira Grande,PT,57.60,38.5167,-28.7000,
17,Marawi,PH,68.95,7.9986,124.2928,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lon = row["Lon"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lon}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotel = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
      hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
    except (IndexError):
      print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
(hotel_df).isnull
hotel_df.dropna()
clean_hotel_df = hotel_df.copy()

In [10]:
clean_hotel_df.head()

,City,Country,Max Temp,Lat,Lon,Hotel Name
0,Taltal,CL,69.75,-25.4000,-70.4833,Residencial Karime
3,Hermanus,ZA,60.80,-34.4187,19.2345,Misty Waves Boutique Hotel
6,Chuy,UY,69.58,-33.6971,-53.4616,Nuevo Hotel Plaza
7,Port Elizabeth,ZA,64.69,-33.9180,25.5701,39 On Nile Guest House
8,Busselton,AU,63.68,-33.6500,115.3333,Observatory Guest House


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lon"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
markers = gmaps.marker_layer(locations)
# 11b. Display the figure
fig = gmaps.figure()
fig

Figure(layout=FigureLayout(height='420px'))